In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project6/lit-master/lit-master')

In [3]:
import argparse
import numpy as np
import tensorflow as tf
from scipy.stats import pearsonr
from utils import *
from neural_network import *
from ensembling_methods import *
from glob import glob


In [4]:

parser = argparse.ArgumentParser()
parser.add_argument("--save_dir", type=str)
parser.add_argument("--n_models", type=int)
parser.add_argument("--dataset", type=str)
parser.add_argument("--split", type=str, default='none')
parser.add_argument('-f')

FLAGS = parser.parse_args()

save_dir = FLAGS.save_dir
n_models = FLAGS.n_models
dataset = FLAGS.dataset
split = FLAGS.split

In [5]:


# Define main helper for evaluating models and saving results
def save_models(models, name, moe_auc=None, moe_val=None, moe_acc=None):
  row = {'ensemble_type': name}
  print(name)
  test_preds = []
  val_preds = []
  accs = []
  aucs = []
  grads = []

  # For each model, save its parameters, compute its individual AUC, and
  # compile its predictions
  for i, m in enumerate(models):
    m.save('{}{}_model{}.pkl'.format(save_dir, name, i))
    testp = m.predict_proba(X_test)
    valp = m.predict_proba(X_val)
    auc = scoring_fun(testp, y_test)
    acc = accuracy_fun(testp, y_test)
    print('  Model #{} AUC: {:.4f}, acc: {:.4f}'.format(i+1,auc,acc))
    aucs.append(auc)
    accs.append(acc)
    test_preds.append(testp)
    val_preds.append(valp)
    grads.append(m.input_gradients(X_test, logits=(y_test.max() == 1)))

  # Save max, min, mean, and standard deviation of individual model AUC
  print('  Indiv AUC max: {:.4f}'.format(np.max(aucs)))
  print('  Indiv AUC min: {:.4f}'.format(np.min(aucs)))
  print('  Indiv AUC mu: {:.4f}'.format(np.mean(aucs)))
  print('  Indiv AUC sd: {:.4f}'.format(np.std(aucs)))

  row['indiv_auc_max'] = '{:.6f}'.format(np.max(aucs))
  row['indiv_auc_min'] = '{:.6f}'.format(np.min(aucs))
  row['indiv_auc_avg'] = '{:.6f}'.format(np.mean(aucs))
  row['indiv_auc_std'] = '{:.6f}'.format(np.std(aucs))

  row['indiv_acc_max'] = '{:.6f}'.format(np.max(accs))
  row['indiv_acc_min'] = '{:.6f}'.format(np.min(accs))
  row['indiv_acc_avg'] = '{:.6f}'.format(np.mean(accs))
  row['indiv_acc_std'] = '{:.6f}'.format(np.std(accs))

  # Compute AUC of average prediction
  val_preds = np.array(val_preds)
  test_preds = np.array(test_preds)
  grads = np.array(grads)
  avg_auc = scoring_fun(test_preds.mean(axis=0), y_test)
  avg_acc = accuracy_fun(test_preds.mean(axis=0), y_test)
  print('  Ens. Avg AUC: {:.4f}, acc: {:.4f}'.format(avg_auc, avg_acc))

  avg_auc_val = scoring_fun(val_preds.mean(axis=0), y_val)

  # Report it (unless it's adaboost and we've been passed its weighted
  # predictions)
  if moe_auc is None:
    row['ensem_auc'] = '{:.6f}'.format(avg_auc)
    row['ensem_acc'] = '{:.6f}'.format(avg_acc)
    row['ensem_val_auc'] = '{:.6f}'.format(avg_auc_val)
  else:
    row['ensem_auc'] = '{:.6f}'.format(moe_auc)
    row['ensem_acc'] = '{:.6f}'.format(moe_acc)
    row['ensem_val_auc'] = '{:.6f}'.format(moe_val)

  # If the ensemble had more than one model (i.e. if it wasn't AdaBoost
  # terminating early), then compute standard diversity measures (+ ours).
  if len(models) > 1:
    # First determine where each model erred
    ens_errors = [error_masks(preds, y_test) for preds in test_preds]
    ens_errsets = [set(np.argwhere(error)[:,0]) for error in ens_errors]

    # Compute the error correlation rho_avg
    error_corr = np.mean([
      pearsonr(err1, err2)[0]
      for i,err1 in enumerate(ens_errors)
      for err2 in ens_errors[i+1:]])

    # Compute the q-statistic
    q_stat = np.mean([
      yules_q_statistic(e1,e2,y_test)
      for i,e1 in enumerate(ens_errsets)
      for e2 in ens_errsets[i+1:]])

    # Compute the interrater agreement (See Eq. 16 of Kuncheva & Whitaker 2003)
    Dis_av = np.mean([
      disagreement_measure(e1,e2,y_test)
      for i,e1 in enumerate(ens_errsets)
      for e2 in ens_errsets[i+1:]])
    avg_acc = np.mean(accs)
    try:
      kappa = 1 - Dis_av / (2 * avg_acc * (1-avg_acc))
    except ZeroDivisionError:
      kappa = np.nan

    # Compute the value of the LIT penalty
    gradcos2 = np.mean([elemwise_sq_cos_sim(g1, g2)
      for i,g1 in enumerate(grads)
      for g2 in grads[i+1:]])

    print('  Q. statistic: {:.4f}'.format(q_stat))
    print('  Interrater agg: {:.4f}'.format(kappa))
    print('  Err. correl.: {:.4f}'.format(error_corr))
    print('  Av grad cos2: {:.4f}'.format(gradcos2))

    row['q_stat'] = '{:.6f}'.format(q_stat)
    row['interrater'] = '{:.6f}'.format(kappa)
    row['err_corr'] = '{:.6f}'.format(error_corr)
    row['grad_cos2'] = '{:.6f}'.format(gradcos2)
  else:
    row['q_stat'] = 'nan'
    row['interrater'] = 'nan'
    row['err_corr'] = 'nan'
    row['grad_cos2'] = 'nan'

  #assert(set(row.keys()) == set(cols))

  # Print everything to the CSV.
  #write_row([row[k] for k in cols])

In [ ]:

for files in glob('/content/drive/MyDrive/Colab Notebooks/project6/lit-master/lit-master/processed_8/processed_8/*'):
    # print(files)
    pc = []
    sd_pc = []
    for items in glob("{}/*.*".format(files)):
        print(items)
        # print(len(items))

        data = np.load(items, allow_pickle=True)
        input_list = data.tolist()  # How to get x_train

        #print(input_list)
        X_train = input_list['FrameStack'][0]
        X = X_train
        X_test = input_list['FrameStack'][1]
        y_train = input_list['FrameStack'][2]
        y_test = input_list['FrameStack'][3]
        X_val = input_list['FrameStack'][4]
        y_val = input_list['FrameStack'][5]

        y_shape = max(y_train)+1

        class Net(NeuralNetwork):
          @property
          def x_shape(self): return [None, X_train.shape[1]]
          @property
          def y_shape(self): return [None, y_shape]
          def rebuild_model(self, X, **_):
            L0 = X
            L1 = tf.compat.v1.layers.dense(L0, 256, name=self.name+'/L1', activation=tf.nn.relu)
            L1 = tf.compat.v1.layers.dropout(L1, training=self.is_train)
            L2 = tf.compat.v1.layers.dense(L1, y_shape, name=self.name+'/L2', activation=None)
            return [L1, L2]

        # Set up training parameters -- we'll use 0.0001 weight decay and train for the
        # minimum epochs to run for 5000 iterations.
        num_epochs = int(np.ceil(np.ceil((5000*128) / float(len(X_train)))))
        train_args = [Net,2, X_train, y_train]

        train_kwargs = {
          'num_epochs': num_epochs,
          'l2_weights': 0.0001,
          'print_every': 100
        }

        # Train random restarts
        tf.compat.v1.reset_default_graph()
        save_models(train_restart_models(*train_args, **train_kwargs), 'restarts')

        # Train bagging
        tf.compat.v1.reset_default_graph()
        save_models(train_bagged_models(*train_args, **train_kwargs), 'baggings')

        # Train adaboost (using scikit-learn's default implementation)
        tf.compat.v1.reset_default_graph()
        adaboost = train_adaboost_models(*train_args, **train_kwargs)
        adaboost_models = [e.mlp for e in adaboost.estimators_]
        save_models(adaboost_models, 'adaboost',
                moe_auc=scoring_fun(adaboost.predict_proba(X_test), y_test),
                moe_val=scoring_fun(adaboost.predict_proba(X_val), y_val),
                moe_acc=accuracy_fun(adaboost.predict_proba(X_test), y_test))

        for penalty in np.logspace(-4, 1, 16):
          # Run LIT
          
          tf.compat.v1.reset_default_graph()
          save_models(
            train_diverse_models(*train_args, lambda_overlap=penalty, **train_kwargs),
            'diverse-{:.4f}'.format(penalty))
          
          # Run NCL
          tf.compat.v1.reset_default_graph()
          save_models(
            train_neg_corr_models(*train_args, lambda_overlap=penalty, **train_kwargs),
            'negcorr-{:.4f}'.format(penalty))

          # Run ACE
          tf.compat.v1.reset_default_graph()
          save_models(
            train_amended_xent_models(*train_args, lambda_overlap=penalty, **train_kwargs),
            'amended-{:.4f}'.format(penalty))

/content/drive/MyDrive/Colab Notebooks/project6/lit-master/lit-master/processed_8/processed_8/Adult_dir/processed_Adult_8.npy
<class '__main__.Net'>


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:401: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs, training=training)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: `tf.layers.dense` is deprecated and will be remov

Batch 0, epoch 0, time 2.1s, accu 0.4531, loss 0.8066, xent 0.8051
Batch 100, epoch 50, time 2.5s, accu 0.8555, loss 0.3174, xent 0.3156
Batch 200, epoch 100, time 2.8s, accu 0.8711, loss 0.2689, xent 0.2669
Batch 300, epoch 150, time 3.1s, accu 0.8984, loss 0.2311, xent 0.2289
Batch 400, epoch 200, time 3.4s, accu 0.9180, loss 0.2199, xent 0.2174
Batch 500, epoch 250, time 3.7s, accu 0.9141, loss 0.2096, xent 0.2068
Batch 600, epoch 300, time 4.0s, accu 0.9062, loss 0.2150, xent 0.2119
Batch 700, epoch 350, time 4.3s, accu 0.9180, loss 0.1783, xent 0.1750
Batch 800, epoch 400, time 4.6s, accu 0.9492, loss 0.1410, xent 0.1375
Batch 900, epoch 450, time 4.9s, accu 0.9297, loss 0.1673, xent 0.1635
Batch 1000, epoch 500, time 5.2s, accu 0.9375, loss 0.1523, xent 0.1483
Batch 1100, epoch 550, time 5.6s, accu 0.9453, loss 0.1393, xent 0.1350
Batch 1200, epoch 600, time 5.9s, accu 0.9219, loss 0.1505, xent 0.1461
Batch 1300, epoch 650, time 6.2s, accu 0.9531, loss 0.1240, xent 0.1193
Batch 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:401: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs, training=training)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: `tf.layers.dense` is deprecated and will be remov

Batch 0, epoch 0, time 1.3s, accu 0.4648, loss 0.7593, xent 0.7577
Batch 100, epoch 50, time 1.6s, accu 0.8750, loss 0.3083, xent 0.3066
Batch 200, epoch 100, time 1.9s, accu 0.8945, loss 0.2675, xent 0.2655
Batch 300, epoch 150, time 2.3s, accu 0.8906, loss 0.2392, xent 0.2370
Batch 400, epoch 200, time 2.6s, accu 0.9062, loss 0.2111, xent 0.2086
Batch 500, epoch 250, time 2.9s, accu 0.9141, loss 0.1864, xent 0.1835
Batch 600, epoch 300, time 3.2s, accu 0.9297, loss 0.1587, xent 0.1556
Batch 700, epoch 350, time 3.5s, accu 0.9219, loss 0.1833, xent 0.1799
Batch 800, epoch 400, time 3.8s, accu 0.9453, loss 0.1554, xent 0.1517
Batch 900, epoch 450, time 4.2s, accu 0.9180, loss 0.1525, xent 0.1486
Batch 1000, epoch 500, time 4.5s, accu 0.9375, loss 0.1564, xent 0.1522
Batch 1100, epoch 550, time 4.8s, accu 0.9375, loss 0.1378, xent 0.1335
Batch 1200, epoch 600, time 5.1s, accu 0.9297, loss 0.1462, xent 0.1416
Batch 1300, epoch 650, time 5.4s, accu 0.9414, loss 0.1141, xent 0.1093
Batch 1

...

Batch 0, epoch 0, time 2.2s, acc1 0.4336, acc2 0.4062, same 0.0020, xent 1.5807
Batch 100, epoch 50, time 2.9s, acc1 0.8867, acc2 0.8711, same 0.0205, xent 0.6168
Batch 200, epoch 100, time 3.6s, acc1 0.8906, acc2 0.9062, same 0.0206, xent 0.5168
Batch 300, epoch 150, time 4.2s, acc1 0.8945, acc2 0.8906, same 0.0196, xent 0.5083
Batch 400, epoch 200, time 4.9s, acc1 0.9180, acc2 0.9219, same 0.0178, xent 0.4045
Batch 500, epoch 250, time 5.6s, acc1 0.9219, acc2 0.9141, same 0.0160, xent 0.3601
Batch 600, epoch 300, time 6.2s, acc1 0.9219, acc2 0.9336, same 0.0145, xent 0.3597
Batch 700, epoch 350, time 6.9s, acc1 0.9258, acc2 0.9453, same 0.0135, xent 0.3287
Batch 800, epoch 400, time 7.6s, acc1 0.9219, acc2 0.9336, same 0.0129, xent 0.3675
Batch 900, epoch 450, time 8.2s, acc1 0.9414, acc2 0.9414, same 0.0122, xent 0.2946
Batch 1000, epoch 500, time 8.9s, acc1 0.9453, acc2 0.9375, same 0.0121, xent 0.2823
Batch 1100, epoch 550, time 9.5s, acc1 0.9258, acc2 0.9180, same 0.0115, xent 0.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:401: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs, training=training)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: `tf.layers.dense` is deprecated and will be remov

Batch 0, epoch 0, time 1.2s, accu 0.5820, loss 0.6813, xent 0.6798
Batch 100, epoch 50, time 1.5s, accu 0.8906, loss 0.2744, xent 0.2726
Batch 200, epoch 100, time 1.8s, accu 0.9102, loss 0.2537, xent 0.2516
Batch 300, epoch 150, time 2.1s, accu 0.9414, loss 0.1811, xent 0.1788
Batch 400, epoch 200, time 2.4s, accu 0.9258, loss 0.1652, xent 0.1626
Batch 500, epoch 250, time 2.8s, accu 0.9336, loss 0.1310, xent 0.1280
Batch 600, epoch 300, time 3.1s, accu 0.9336, loss 0.1508, xent 0.1475
Batch 700, epoch 350, time 3.4s, accu 0.9531, loss 0.1379, xent 0.1344
Batch 800, epoch 400, time 3.7s, accu 0.9531, loss 0.1073, xent 0.1035
Batch 900, epoch 450, time 4.1s, accu 0.9570, loss 0.1054, xent 0.1013
Batch 1000, epoch 500, time 4.4s, accu 0.9453, loss 0.1216, xent 0.1173
Batch 1100, epoch 550, time 4.7s, accu 0.9688, loss 0.0999, xent 0.0953
Batch 1200, epoch 600, time 5.0s, accu 0.9648, loss 0.0780, xent 0.0731
Batch 1300, epoch 650, time 5.3s, accu 0.9688, loss 0.0701, xent 0.0651
Batch 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:401: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs, training=training)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: `tf.layers.dense` is deprecated and will be remov

Batch 0, epoch 0, time 2.0s, acc1 0.4102, acc2 0.6016, same 0.0023, xent 1.5121
Batch 100, epoch 50, time 2.7s, acc1 0.9062, acc2 0.9023, same 0.0210, xent 0.5562
Batch 200, epoch 100, time 3.3s, acc1 0.8945, acc2 0.9141, same 0.0220, xent 0.4649
Batch 300, epoch 150, time 4.0s, acc1 0.9141, acc2 0.9023, same 0.0220, xent 0.4262
Batch 400, epoch 200, time 4.6s, acc1 0.9180, acc2 0.9180, same 0.0222, xent 0.3732
Batch 500, epoch 250, time 5.2s, acc1 0.9297, acc2 0.9375, same 0.0220, xent 0.3057
Batch 600, epoch 300, time 5.9s, acc1 0.9492, acc2 0.9414, same 0.0218, xent 0.3072
Batch 700, epoch 350, time 6.5s, acc1 0.9492, acc2 0.9336, same 0.0216, xent 0.2429
Batch 800, epoch 400, time 7.2s, acc1 0.9531, acc2 0.9609, same 0.0214, xent 0.2135
Batch 900, epoch 450, time 7.8s, acc1 0.9492, acc2 0.9609, same 0.0211, xent 0.2054
Batch 1000, epoch 500, time 8.4s, acc1 0.9766, acc2 0.9648, same 0.0208, xent 0.1761
Batch 1100, epoch 550, time 9.1s, acc1 0.9688, acc2 0.9688, same 0.0206, xent 0.